## Techniques I tried

In [1]:
import warnings 
  
warnings.filterwarnings(action = 'ignore') 
  
import gensim 
from gensim.models import Word2Vec 

In [ ]:
import pandas as pd
import numpy as np
import nltk, json, re
from nltk.corpus import stopwords 

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
stop_words = set(stopwords.words('english')) 

In [ ]:
train_data = pd.read_csv("/home/hackerearth/Desktop/ml_comptetion/dataset/train.csv")
test_data = pd.read_csv("/home/hackerearth/Desktop/ml_comptetion/dataset/test.csv")
with open("/home/hackerearth/Downloads/contractions.json", "r") as file:
    contractions = json.load(file)

### Embedding using BERT  as a service : (This code is not including in the file because it is running in the local server of bert in cmd)
steps :-
a: pip install tensorflow==1.15.0
b: pip install -U bert-serving
c: bert-serving-start - model_dir -num_workers
Meanwhile in python:
from bert_serving.client import BertClient
b = BertClient()
b.encode(["string"])

After getting embedding from here I tried many classifiers like Logistic regresssion, Random Forest, XGBoost, Decision tree with their respective grid search.

In [4]:
print(train_data.shape)
train_data.head()

(3235, 6)


,id,original_text,lang,retweet_count,original_author,sentiment_class
0,1.245025e+18,Happy #MothersDay to all you amazing mothers o...,en,0,BeenXXPired,0
1,1.245759e+18,Happy Mothers Day Mum - I'm sorry I can't be t...,en,1,FestiveFeeling,0
2,1.246087e+18,Happy mothers day To all This doing a mothers ...,en,0,KrisAllenSak,-1
3,1.244803e+18,Happy mothers day to this beautiful woman...ro...,en,0,Queenuchee,0
4,1.244876e+18,Remembering the 3 most amazing ladies who made...,en,0,brittan17446794,-1


In [5]:
print(test_data.shape)
test_data.head()

(1387, 5)


,id,original_text,lang,retweet_count,original_author
0,1.246628e+18,"3. Yeah, I once cooked potatoes when I was 3 y...",en,0,LToddWood
1,1.245898e+18,"Happy Mother's Day to all the mums, step-mums,...",en,0,iiarushii
2,1.244717e+18,"I love the people from the UK, however, when I...",en,0,andreaanderegg
3,1.245730e+18,Happy 81st Birthday Happy Mother’s Day to my m...,en,1,TheBookTweeters
4,1.244636e+18,Happy Mothers day to all those wonderful mothe...,en,0,andreaanderegg


In [6]:
train_data["sentiment_class"].value_counts()

 0    1701
-1     769
 1     765
Name: sentiment_class, dtype: int64

## Preprocessing:
a: remove urls
b: punctuations
c: stopwords 
d: lemmitization

In [7]:
def remove_url(data):
    temp = []
    for i in data:
        i = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))|https''', '', i, flags=re.MULTILINE)
        #i = re.sub(r'^https?|:\/\/.*[\r\n]*|https', '', i, flags=re.MULTILINE)
        temp.append(i)
    return temp

In [8]:
train = remove_url(train_data["original_text"])
test = remove_url(test_data["original_text"])

In [9]:
# clean punctuation
def clean_punctuation(data):
    punctuation = re.compile(r'[-.?!,#@=$%^&*_\><":;()|0-9]')
    temp =[]
    for i in data:
        temp.append([punctuation.sub("",j) for j in i.split(" ") 
                     if j not in contractions.keys() and
                     punctuation.sub("",j) not in stop_words and 
                    len(punctuation.sub("",j))>1])
    return temp

In [10]:
train = clean_punctuation(train)
test = clean_punctuation(test)

In [16]:
with open("/home/hackerearth/Desktop/ml_comptetion/dataset/train.txt",'w') as file:
    for listitem in train:
        file.write('%s\n' % listitem)
with open("/home/hackerearth/Desktop/ml_comptetion/dataset/test.txt",'w') as file:
    for listitem in train:
        file.write('%s\n' % listitem)

In [ ]:
data = train[:]
data.extend(test)

## Word2Vec inclusing many classifier 

In [12]:
model = gensim.models.Word2Vec(data, min_count = 1,  
                              size = 1024, window = 5) 

In [242]:
def embedding(data):
    temp =[]
    for i in data:
        l = []
        for j in i:
            l.append(np.reshape(model[j],(1,1024)))
        l=np.average(np.array(l),axis=0)
        temp.append(l)
    return temp
train_emb = embedding(train)
test_emb = np.array(embedding(test))

In [7]:
#train_emb = np.concatenate(train_emb, axis=0)
#test_emb = np.concatenate(test_emb, axis=0)

#............. Bert embeddings ..............#

import pickle
train_emb = pickle.load(open("/home/hackerearth/Desktop/ml_comptetion/dataset/train2.npy",'rb'))
test_emb = pickle.load(open("/home/hackerearth/Desktop/ml_comptetion/dataset/test2.npy",'rb'))

In [8]:
train_emb = np.reshape(train_emb,(3235,1024))
test_emb = np.reshape(test_emb,(1387,1024))
y_labels = train_data["sentiment_class"]

### Logistic regression

In [13]:
def LR_model(train_emb, y_labels):
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import accuracy_score
    a =[]
    for i in range(10):
        x_train, x_test, y_train, y_test = train_test_split(train_emb, y_labels, train_size=0.8)
        model = LogisticRegression(solver = 'lbfgs')
        model.fit(x_train,y_train)
        predict_label = model.predict(x_test)
        a.append(accuracy_score(predict_label, y_test))
    print(np.array(a).mean())
    return model

In [14]:
model = LR_model(train_emb, y_labels)
prediction = model.predict(test_emb)

result = pd.DataFrame(prediction, test_data["id"], columns=["sentiment_class"])
result.to_csv("/home/hackerearth/Desktop/ml_comptetion/dataset/sample.csv")

0.43168469860896447


In [46]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
x_train, x_test, y_train, y_test = train_test_split(train_emb, y_labels, train_size=0.8)
grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
logreg=LogisticRegression()
logreg_cv=GridSearchCV(logreg,grid,cv=10)
logreg_cv.fit(x_train,y_train)

print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

tuned hpyerparameters :(best parameters)  {'C': 0.001, 'penalty': 'l2'}
accuracy : 0.531298075484122


In [50]:
logreg2=LogisticRegression(C=0.001,penalty="l2")
logreg2.fit(train_emb,y_labels)
prediction = logreg2.predict(test_emb)

result = pd.DataFrame(prediction, test_data["id"], columns=["sentiment_class"])
result.to_csv("/home/hackerearth/Desktop/ml_comptetion/dataset/sample.csv")

## LR + somte

In [15]:
def LR_model_smote(train_emb, y_labels):
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import accuracy_score
    from imblearn.over_sampling import SMOTE
    a =[]
    smote = SMOTE('minority')
    x_train, x_test, y_train, y_test = train_test_split(train_emb, y_labels, train_size=0.8)
    x_train, y_train = smote.fit_sample(x_train, y_train)
    model = LogisticRegression(solver = 'lbfgs')
    model.fit(x_train,y_train)
    predict_label = model.predict(x_test)
    a.append(accuracy_score(predict_label, y_test))
    print(np.array(a).mean())
    return model

In [16]:
model = LR_model_smote(train_emb, y_labels)
prediction = model.predict(test_emb)

Using TensorFlow backend.


0.39258114374034003


In [17]:
result = pd.DataFrame(prediction, test_data["id"], columns=["sentiment_class"])
result.to_csv("/home/hackerearth/Desktop/ml_comptetion/dataset/sample.csv")

# Keras

In [18]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

In [19]:
encoder = LabelEncoder()
encoder.fit(y_labels)
encoded_Y = encoder.transform(y_labels)
dummy_y = np_utils.to_categorical(encoded_Y)

In [20]:
input_dim = train_emb.shape[1]

model = Sequential()
model.add(Dense(512, input_dim = input_dim , activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(3, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )

model.fit(train_emb, dummy_y, epochs = 50, batch_size = 5)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Epoch 1/50
3235/3235 [==============================] - 11s 3ms/step - loss: 1.0552 - accuracy: 0.5202
Epoch 2/50
3235/3235 [==============================] - 10s 3ms/step - loss: 1.0246 - accuracy: 0.5243
Epoch 3/50
3235/3235 [==============================] - 9s 3ms/step - loss: 1.0232 - accuracy: 0.5258
Epoch 4/50
3235/3235 [==============================] - 8s 3ms/step - loss: 1.0243 - accuracy: 0.5255
Epoch 5/50
3235/3235 [==============================] - 8s 2ms/step - loss: 1.0258 - accuracy: 0.5215
Epoch 6/50
3235/3235 [==============================] - 11s 3ms/step - loss: 1.0215 - accuracy: 0.5270
Epoch 7/50
3235/3235 [==============================] - 9s 3ms/step - loss: 1.0220 - accuracy: 0.5243
Epoch 8/50
3235/3235 [==============================] - 10s 3ms/step - loss: 1.0163 - accuracy: 0.5277
Epoch 9/50
3235/3235 [==============================] - 9s 3ms/step - loss: 1.0126 - accuracy: 0.5

In [21]:
prediction = model.predict_classes(test_emb)
result = pd.DataFrame(prediction, test_data["id"], columns=["sentiment_class"])
result.to_csv("/home/hackerearth/Desktop/ml_comptetion/dataset/sample.csv")

# Decision tree

In [31]:
def decision_tree():
    from sklearn.tree import DecisionTreeClassifier
    x_train, x_test, y_train, y_test = train_test_split(train_emb, y_labels, train_size=0.8)
    parameters = {'min_samples_split' : range(10,500,20),'max_depth': range(1,20,2)}
    clf_tree = DecisionTreeClassifier()
    clf = GridSearchCV(clf_tree ,parameters)
    clf.fit(x_train, y_train)
    pred = clf.predict(test_emb)
    return pred

In [32]:
pred = decision_tree()
result = pd.DataFrame(pred, test_data["id"], columns=["sentiment_class"])
result.to_csv("/home/hackerearth/Desktop/ml_comptetion/dataset/sample.csv")

# SVC

In [33]:
def scs():
    from sklearn.svm import SVC 
    x_train, x_test, y_train, y_test = train_test_split(train_emb, y_labels, train_size=0.8)
    param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
    grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
    grid.fit(x_train, y_train) 
    prediction = grid.predict(test_emb)
    return prediction

In [34]:
pred = scs()
result = pd.DataFrame(pred, test_data["id"], columns=["sentiment_class"])
result.to_csv("/home/hackerearth/Desktop/ml_comptetion/dataset/sample.csv")

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.525, total=  16.2s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.2s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.525, total=  16.1s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   32.3s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.525, total=  16.3s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.526, total=  16.2s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.526, total=  16.1s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.525, total=  16.3s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.525, total=  16.4s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.525, total=  16.3s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.526, total=  16.0s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] .

[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.463, total=  16.2s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.454, total=  16.1s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.442, total=  16.6s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.445, total=  16.3s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.439, total=  17.3s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=0.527, total=  13.5s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=0.525, total=  13.7s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] .

[CV] ..... C=1000, gamma=0.001, kernel=rbf, score=0.412, total=  17.8s
[CV] C=1000, gamma=0.001, kernel=rbf .................................
[CV] ..... C=1000, gamma=0.001, kernel=rbf, score=0.395, total=  17.5s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.446, total=  13.3s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.454, total=  13.6s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.415, total=  13.3s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.447, total=  13.6s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] .... C=1000, gamma=0.0001, kernel=rbf, score=0.441, total=  13.4s


[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed: 31.7min finished


# Random forest

In [27]:
# random forest
from sklearn.ensemble import RandomForestClassifier    
x_train, x_test, y_train, y_test = train_test_split(train_emb, y_labels, train_size=0.8)
param_grid = { 
    'n_estimators': [10, 5, 8, 15, 20],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}
classifier = RandomForestClassifier(random_state = 42)
CV_rfc = GridSearchCV(estimator=classifier, param_grid=param_grid, cv= 5)
CV_rfc.fit(x_train, y_train)
CV_rfc.best_params_    

{'criterion': 'entropy',
 'max_depth': 4,
 'max_features': 'log2',
 'n_estimators': 10}

In [28]:
rfc1=RandomForestClassifier(max_features='log2', n_estimators= 10, 
                            max_depth=4, criterion='entropy')
rfc1.fit(x_train, y_train)
pred = rfc1.predict(test_emb)
result = pd.DataFrame(pred, test_data["id"], columns=["sentiment_class"])
result.to_csv("/home/hackerearth/Desktop/ml_comptetion/dataset/sample.csv")

# XGBOOST

In [22]:
# XGB classifier
import xgboost as xgb
x_train, x_test, y_train, y_test = train_test_split(train_emb, y_labels, train_size=0.8)
clf = xgb.XGBClassifier(max_depth=5, objective='multi:softmax', n_estimators=1000, 
                        num_classes=9)
clf.fit(x_train, y_train)  
pred = clf.predict(test_emb)
result = pd.DataFrame(pred, test_data["id"], columns=["sentiment_class"])
result.to_csv("/home/hackerearth/Desktop/ml_comptetion/dataset/sample.csv")

[10:53:51] WARNING: /workspace/src/learner.cc:480: 
Parameters: { num_classes } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [30]:
# gradient boosting 
from sklearn.ensemble import GradientBoostingClassifier
x_train, x_test, y_train, y_test = train_test_split(train_emb, y_labels, train_size=0.8)
clf = GradientBoostingClassifier(n_estimators=20, learning_rate=0.8
                                 , max_features=2, max_depth=2, random_state=0)
clf.fit(x_train, y_train)
pred = clf.predict(test_emb)
result = pd.DataFrame(pred, test_data["id"], columns=["sentiment_class"])
result.to_csv("/home/hackerearth/Desktop/ml_comptetion/dataset/sample.csv")

In [ ]:
param_test1 = {'n_estimators':range(20,81,10)}
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1,
                                                               min_samples_split=500,
                                                               min_samples_leaf=50,
                                                               max_depth=8,max_features='sqrt',
                                                               subsample=0.8,random_state=10),param_grid = param_test1, 
                                                               scoring='roc_auc',n_jobs=4,iid=False, cv=5)
x_train, x_test, y_train, y_test = train_test_split(train_emb, y_labels, train_size=0.8)
gsearch1.fit(x_train, y_train)
pred = gsearch1.predict(test_emb)
result = pd.DataFrame(pred, test_data["id"], columns=["sentiment_class"])
result.to_csv("/home/hackerearth/Desktop/ml_comptetion/dataset/sample.csv")